In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from itertools import product
import scipy.stats as ss
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Loading Dataset

In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/AI Hackathon /Convolve /Round 2/Copy of FileAndResults_Problem2.xlsx')

In [ ]:
df.head(5)

,File Changed,Component,Coder,Tester,Category,Sanity name,nets Tested,Sanity Result
0,cpp/csyu/category/fnf,asr1001-flowpsk,Batukbaj,samsk,PD-asr1001,Thor_Vista,NCS5500,Pass
1,ignor-dp/build,asr1001-ignor,Alex,manishakang,Tools,vistra-cxr,NCS5500,Aborted
2,ignor-dp/cdp,asr1001-ignor,Alex,manishakang,PD-asr1001,vistra-cxr,NCS5500,Fail
3,ignor-dp/csco/affinity,asr1001-ignor,Alex,manishakang,PD-asr1001,vistra-cxr,NCS5500,Aborted
4,ignor-dp/csco/cdp,asr1001-ignor,Alex,manishakang,PD-asr1001,vistra-cxr,NCS5500,Aborted


In [ ]:
df.describe()

,File Changed,Component,Coder,Tester,Category,Sanity name,nets Tested,Sanity Result
count,351,351,351,337,351,351,351,351
unique,176,45,7,18,11,25,14,3
top,cpp/csyu/category/fnf,asr1001-forge-cgn,patrick,manishakang,PD-asr1001,vistra-cxr,"asr1001-PX,asr1001-X64",Pass
freq,2,70,130,109,146,122,105,210


#Data Pre-processing



In [ ]:
#Plotting Correlation Matrices
headers= ('File Changed', 'Component', 'Coder', 'Tester', 'Category','Sanity name', 'nets Tested', 'Sanity Result')
cat_var_prod = list(product(headers,headers, repeat = 1))
result = []
for i in cat_var_prod:
  if i[0] != i[1]:
    result.append((i[0],i[1],list(ss.chi2_contingency(pd.crosstab(df[i[0]], df[i[1]])))[1]))

chi_test_output = pd.DataFrame(result, columns = ['var1','var2','coeff'])
chi_test_output.pivot(index='var1',columns='var2',values='coeff')

var2,Category,Coder,Component,File Changed,Sanity Result,Sanity name,Tester,nets Tested
var1,,,,,,,,
Category,NaN,7.804164e-42,4.826306e-183,2.708323e-120,7.636152e-14,1.330431e-301,2.032295e-184,2.928507e-282
Coder,7.804164e-42,NaN,2.517941e-217,2.342799e-52,1.064056e-07,3.929845e-118,3.710492e-184,5.548302e-65
Component,4.826306e-183,2.517941e-217,NaN,0.000000e+00,1.252796e-26,0.000000e+00,0.000000e+00,5.347049e-199
File Changed,2.708323e-120,2.342799e-52,0.000000e+00,NaN,5.131404e-20,2.964803e-285,1.303508e-175,1.126545e-115
Sanity Result,7.636152e-14,1.064056e-07,1.252796e-26,5.131404e-20,NaN,4.973096e-14,3.724433e-33,1.798112e-10
Sanity name,1.330431e-301,3.929845e-118,0.000000e+00,2.964803e-285,4.973096e-14,NaN,0.000000e+00,2.104017e-298
Tester,2.032295e-184,3.710492e-184,0.000000e+00,1.303508e-175,3.724433e-33,0.000000e+00,NaN,1.023524e-164
nets Tested,2.928507e-282,5.548302e-65,5.347049e-199,1.126545e-115,1.798112e-10,2.104017e-298,1.023524e-164,NaN


In [ ]:
#Removing "/" from "File changed" i.e changing "ignor-dp/csco/logrotate" to "ignor-dp csco logrotate".
dir = list(df['File Changed'])
new_dir = []
for i in dir:
  j = i.replace("/"," ")
  new_dir.append(j)
df['File Changed'] = new_dir

In [ ]:
df.head(5)

,File Changed,Component,Coder,Tester,Category,Sanity name,nets Tested,Sanity Result
0,cpp csyu category fnf,asr1001-flowpsk,Batukbaj,samsk,PD-asr1001,Thor_Vista,NCS5500,Pass
1,ignor-dp build,asr1001-ignor,Alex,manishakang,Tools,vistra-cxr,NCS5500,Aborted
2,ignor-dp cdp,asr1001-ignor,Alex,manishakang,PD-asr1001,vistra-cxr,NCS5500,Fail
3,ignor-dp csco affinity,asr1001-ignor,Alex,manishakang,PD-asr1001,vistra-cxr,NCS5500,Aborted
4,ignor-dp csco cdp,asr1001-ignor,Alex,manishakang,PD-asr1001,vistra-cxr,NCS5500,Aborted


In [ ]:
headers=['File Changed','Component','Coder','Tester','Category','Sanity name','nets Tested','Sanity Result']
count=[176,45,7,18,11,25,14,3]

#One hot Encoding
for i in range(2,8):
    if(i!=5):
        df[headers[i]]=df[headers[i]].astype('category')
        df[headers[i]]=df[headers[i]].cat.codes
        enc=OneHotEncoder()
        enc_data=pd.DataFrame(enc.fit_transform(df[[headers[i]]]).toarray())
        New_df=df.join(enc_data)
        New_df.drop(list(range(0,count[i])),axis=1)
        df.drop([headers[i]],axis=1)

In [ ]:
# Function for Bag of Words
def bag_of_word(name,df,a,b):
  new_dir1=df[name]
  cv = CountVectorizer()
  X = cv.fit_transform(new_dir1)
  sorted(cv.vocabulary_.keys())
  n=len(cv.vocabulary_.keys())
  X = X.toarray()
  f = list(range(a,b))
  f = map(str,f)
  data = pd.DataFrame(X, columns=f )
  return data,n

In [ ]:
#Changing "File Change","Component","Snaity name" using Bag of words. 
data1,n=bag_of_word('File Changed',df,1,137)
data2,n=bag_of_word('Component',df,137,173)
data3,n=bag_of_word('Sanity name',df,173,198)
frames = [data1,data2,data3,df]
result = pd.concat(frames, axis=1, join='inner')
df=result.drop(['File Changed','Component','Sanity name'],axis=1)
df.head(5)

,1,2,3,4,5,6,7,8,9,10,...,193,194,195,196,197,Coder,Tester,Category,nets Tested,Sanity Result
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,12,4,1,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,7,9,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,7,4,1,1
3,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,7,4,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,7,4,1,0


In [ ]:
y=df['Sanity Result']
X=df.drop(['Sanity Result'],axis=1)

#Splitting Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Training Phase

In [ ]:
#Random Forest
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit(X_train,y_train)
r=forest.predict(X_test)
probab = forest.predict_proba(X_test)
cm = confusion_matrix(y_test,r)
print(metrics.accuracy_score(y_test, r)*100)
cm

94.36619718309859


array([[14,  0,  2],
       [ 0, 11,  1],
       [ 0,  1, 42]])

In [ ]:
#LogisticRegression 
model = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model.fit(X_train, y_train)
r=model.predict(X_test)
cm = confusion_matrix(y_test,r)
print(metrics.accuracy_score(y_test, r)*100)
cm

87.32394366197182


array([[14,  0,  2],
       [ 0, 10,  2],
       [ 2,  3, 38]])

In [ ]:
#DecisionTree
dtree = DecisionTreeClassifier()
dtree = dtree.fit(X_train, y_train)
r=dtree.predict(X_test)
cm = confusion_matrix(y_test,r)
print(metrics.accuracy_score(y_test, r)*100)
cm

90.14084507042254


array([[12,  0,  4],
       [ 1, 11,  0],
       [ 1,  1, 41]])

In [ ]:
#GaussianNB
gnb = GaussianNB()
a = gnb.fit(X_train, y_train)  
y_pred = a.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
print(metrics.accuracy_score(y_test, y_pred)*100)
cm

80.28169014084507


array([[16,  0,  0],
       [ 2, 10,  0],
       [ 6,  6, 31]])

In [ ]:
#XGBoostClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
print(metrics.accuracy_score(y_test, y_pred)*100)
cm

74.64788732394366


array([[ 5,  1, 10],
       [ 0,  8,  4],
       [ 2,  1, 40]])

In [ ]:
#Proposed model
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
m1 = LogisticRegression(multi_class='multinomial', solver='newton-cg')
m2 = RandomForestClassifier(n_estimators = 100)
eclf1 = VotingClassifier(estimators=[('l1', m1), ('l2',m2)], voting='soft')
eclf1 = eclf1.fit(X_train, y_train)
r = eclf1.predict(X_test)
cm = confusion_matrix(y_test,r)
print(metrics.accuracy_score(y_test,r)*100)
cm

97.1830985915493


array([[16,  0,  0],
       [ 0, 11,  1],
       [ 0,  1, 42]])

In [ ]:
#Finding probabilty of "Fail","Pass" and "Aborted"
l1=[]
l2=[]
l3=[]
for i in probab:
  l1.append(i[0])
  l2.append(i[1])
  l3.append(i[2])
X_test['Prob_Aborted']=l1
X_test['Prob_Fail']=l2
X_test['Prob_Pass']=l3
# X_test['Predicted']=r
l4=[]
for i in r:
  if(i==0):
    l4.append('Aborted')
  elif(i==1):
    l4.append('Fail')
  else:
    l4.append('Pass')
X_test['Predicted']=l4
X_test

,1,2,3,4,5,6,7,8,9,10,...,196,197,Coder,Tester,Category,nets Tested,Prob_Aborted,Prob_Fail,Prob_Pass,Predicted
157,0,0,0,0,0,0,0,0,0,0,...,0,0,6,7,4,10,0.68,0.05,0.27,Aborted
342,0,0,0,0,0,0,0,0,0,0,...,0,0,0,7,4,10,0.17,0.29,0.54,Pass
316,0,0,0,0,0,0,0,0,0,0,...,0,0,6,7,4,10,0.04,0.54,0.42,Fail
234,0,0,0,0,0,0,0,0,0,0,...,0,0,3,1,6,2,0.01,0.04,0.95,Pass
155,0,0,0,0,0,0,0,0,0,0,...,0,0,6,7,4,12,0.07,0.25,0.68,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,0,0,0,0,0,0,0,0,0,0,...,0,0,6,7,10,1,0.39,0.06,0.55,Aborted
179,0,0,0,0,0,0,0,0,0,0,...,0,0,6,7,4,11,0.50,0.17,0.33,Aborted
199,0,0,0,0,0,0,0,0,0,0,...,1,0,2,6,2,10,0.03,0.11,0.86,Pass
327,0,0,0,0,0,0,0,0,0,0,...,0,0,6,7,4,10,0.03,0.02,0.95,Pass
